In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime, timedelta
from scipy.stats import poisson
from random import randrange
import pandas as pd


In [ ]:
class PoissonGenerator:
    """
    Generate transactions timestamps list. Current version works with milliseconds.
    """
    
    def __init__(self, cycle_size: int, mean_occurencies: float):
        """
        Create a Poisson generator
        
        Keyword_arguments:
        cycle_size (int) -- reviewable cycle size in milliseconds
        mean_occurencies (float) -- mean amount of transactions
            happening in given cycle size
        """
        self.cycle_size = cycle_size
        self.mean_occurencies = mean_occurencies
        self.cumulative_probabilities = 0
        
        
    def __generate_poisson_outcome__(self) -> int:
        """
        Generate how many transactions will happen, considering cycle size 
        and mean transaction amount
        """
        return np.random.poisson(self.mean_occurencies, 1)[0]
    
    
    def generate_transactions(self, current_timestamp: datetime) -> list:
        """
        Generate array of transactions timestamps, considering Poisson distribution and
        random distribution of timestamps in given cycle size
        
        Keyword_arguments:
        current_timestamp (datetime) -- reviewable cycle starting point
        """
        transactions_timestamps = []
        current_transaction_count = self.__generate_poisson_outcome__()
        
        for i in range(current_transaction_count):
            transactions_timestamps.append(current_timestamp + timedelta(milliseconds=randrange(self.cycle_size)))
        
        return transactions_timestamps
    

In [ ]:
class NormalGenerator:
    """
    Generate transaction values conform normal distribution
    """
    
    def __init__(self, mu: float=0, sigma: float=0):
        """
        Create a normal distribution generator
        
        Keyword_arguments:
        mu (float) -- mean transaction price (default 0.0)
        sigma (float) -- standard deviation of transaction price (default 0.0)
        """
        self.mu = mu
        self.sigma = sigma
      
    
    def reset_params(self, mu: float, sigma: float):
        """
        Change parameters of existing generator
        
        Keyword_arguments:
        mu (float) -- mean transaction price (default 0.0)
        sigma (float) -- standard deviation of transaction price (default 0.0)
        """
        self.mu = mu
        self.sigma = sigma
    

    def generate_transactions(self, transactions_count: int) -> list:
        """
        Generate transaction prices list considering mu and sigma
        
        Keyword_arguments:
        transactions_count (int) -- required transactions count
        """
        return np.random.normal(self.mu, self.sigma, transactions_count)


In [ ]:
class Transaction:
    def __init__(self, id: int, timestamp: datetime, value: float):
        self.id = id
        self.timestamp = timestamp
        self.value = value
        
    
    def to_string(self) -> str:
        return str('Transaction {id = ' + str(self.id) + ', timestamp = "' + str(self.timestamp) + '", value = ' + str(self.value)) + '}'

In [ ]:
class MonteCarloTransactionSimulator:
    """
    Monte Carlo transactions generator, that generates transactions frequency using Poisson 
    distribution and transaction values using normal distribution (time metrics - milliseconds)
    """
    def __init__(
        self, transaction_density_generator: PoissonGenerator, transaction_values_generator: NormalGenerator, 
        first_currency: str, second_currency: str
    ):
        """
        Create a Monte-Carlo transaction simulator
        
        Keyword_arguments:
        transaction_density_generator (PoissonGenerator) -- Poisson transaction distribution generator
        transaction_values_generator (NormalGenerator) -- Normal transaction distribution generator
        """
        self.transaction_density_generator = transaction_density_generator
        self.transaction_values_generator = transaction_values_generator
        self.first_currency = first_currency
        self.second_currency = second_currency
        self.transaction_history = []
        
    
    def reset_values_generator_params(self, mu: float, sigma: float):
        """
        Change parameters of existing values (normal distribution) generator
        
        Keyword_arguments:
        mu (float) -- mean transaction price (default 0.0)
        sigma (float) -- standard deviation of transaction price (default 0.0)
        """
        self.transaction_density_generator.reset_params(mu, sigma)
        
        
    def clear_transaction_history(self):
        """
        Clear all records from transaction history
        """
        self.transaction_history.clear()
    
    
    def generate_transactions(self, current_timestamp: datetime):
        """
        Generates transactions list with timestamps and values assigned to 
        
        Keyword arguments:
        current_timestamp (datetime) -- initial datetime point from where cycle will be reviewed
        """
        # generate timestamps (and sort them) and generate transaction values
        timestamps = self.transaction_density_generator.generate_transactions(current_timestamp)
        timestamps.sort()
        values = self.transaction_values_generator.generate_transactions(len(timestamps))
        
        # create transactions, appending them to the transaction history
        for index in range(len(timestamps)):
            new_transaction = Transaction(index, timestamps[index], values[index])
            self.transaction_history.append(new_transaction)
            
            
    def transaction_history_to_csv(self, filename: str):
        """
        Write transaction history to specified .csv file
        
        Keyword arguments:
        filename (str) -- name of .csv file where to write data
        """
        # form empty dataframe
        transaction_history_dataframe = pd.DataFrame(columns=['id', 'FirstCurrency', 'SecondCurrency', 'Timestamp', 'Value'])
        
        # append all new records to the dataframe
        for index in range(len(self.transaction_history)):
            new_row = {
                'id': self.transaction_history[index].id, 
                'FirstCurrency': self.first_currency, 
                'SecondCurrency': self.second_currency, 
                'Timestamp': self.transaction_history[index].timestamp,
                'Value': self.transaction_history[index].value
            }
            transaction_history_dataframe = transaction_history_dataframe.append(new_row, ignore_index=True)
        
        # if there is such file -> append new records to it, otherwise create a new file from existing table
        try:
            with open(filename) as f:
                transaction_history_dataframe.to_csv(filename, mode='a', header=False)
        except IOError:
            transaction_history_dataframe.to_csv(filename)

In [ ]:
# example of creating simulation
simulator = MonteCarloTransactionSimulator(
    PoissonGenerator(cycle_size=60000, mean_occurencies=5), 
    NormalGenerator(mu=10, sigma=5), 'ETH', 'DAI'
)

#  set starting point to be as current timestamp and then start loop, where after each iteration 
# reviewable timestamp will be updated by shifting it further conform generator cycle size
current_iteration_timestamp = datetime.now()
for index in range(60*24*7):
    simulator.generate_transactions(current_iteration_timestamp)
    current_iteration_timestamp += timedelta(milliseconds=randrange(simulator.transaction_density_generator.cycle_size))

# show all generated transactions
for index in range(len(simulator.transaction_history)):
    print(simulator.transaction_history[index].to_string())
    
# write all generated transactions (entire generated transaction history) to csv file
simulator.transaction_history_to_csv('history.csv')

# Next version of the generator, applying additional details to the generator work principle

In [ ]:
class NormalDeviationFactorGenerator:
    """
    Generate deviation coefficient factor of the actual price from AMM-estimated one
    """
    
    def __init__(self, matching_coefficient: float=1, deviation_factor: float=0):
        """
        Create a normal distribution generator
        
        Keyword_arguments:
        matching_coefficient (float) -- how AMM price matches with actual price
            on price at the moment of transaction execution (default 1.0)
        deviation_factor (float) -- standard deviation of the matching coefficient
            (default 0.0)
        """
        self.matching_coefficient = matching_coefficient
        self.deviation_factor = deviation_factor
      
    
    def reset_params(self, matching_coefficient: float, deviation_factor: float):
        """
        Change parameters of existing generator
        
        Keyword_arguments:
        matching_coefficient (float) -- how AMM price matches with actual price
            on price at the moment of transaction execution (default 1.0)
        deviation_factor (float) -- standard deviation of the matching coefficient
            (default 0.0)
        """
        self.matching_coefficient = matching_coefficient
        self.deviation_factor = deviation_factor


    def generate_transaction_multiplier(self) -> float:
        """
        Generate transaction multiplier for current transaction conform mean
        matching coefficient and deviation factor
        """
        return np.random.normal(self.matching_coefficient, self.deviation_factor, 1)[0]



deviation_generator = NormalDeviationFactorGenerator(matching_coefficient=1, deviation_factor=0.01)
for index in range(100):
    print(str(100 * deviation_generator.generate_transaction_multiplier()))


# cooperation with Dima

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime, timedelta
from scipy.stats import poisson, truncnorm
from random import randrange
import pandas as pd


class PoissonGenerator:
    """
    Generate transactions timestamps list. Current version works with milliseconds.
    """
    
    def __init__(self, cycle_size: int, mean_occurencies: float):
        """
        Create a Poisson generator
        
        Keyword_arguments:
        cycle_size (int) -- reviewable cycle size in milliseconds
        mean_occurencies (float) -- mean amount of transactions
            happening in given cycle size
        """
        self.cycle_size = cycle_size
        self.mean_occurencies = mean_occurencies
        self.cumulative_probabilities = 0
        
        
    def __generate_poisson_outcome__(self) -> int:
        """
        Generate how many transactions will happen, considering cycle size 
        and mean transaction amount
        """
        return np.random.poisson(self.mean_occurencies, 1)[0]
    
    
    def generate_transactions(self, current_timestamp: datetime) -> list:
        """
        Generate array of transactions timestamps, considering Poisson distribution and
        random distribution of timestamps in given cycle size
        
        Keyword_arguments:
        current_timestamp (datetime) -- reviewable cycle starting point
        """
        transactions_timestamps = []
        current_transaction_count = self.__generate_poisson_outcome__()
        
        for i in range(current_transaction_count):
            transactions_timestamps.append(current_timestamp + timedelta(milliseconds=randrange(self.cycle_size)))
        
        return transactions_timestamps
    


class NormalGenerator:
    """
    Generate transaction values conform normal distribution
    """
    
    def __init__(self, mu: float, sigma: float, lower_bound: float, upper_bound: float):
        """
        Create a normal distribution generator
        
        Keyword_arguments:
        mu (float) -- mean transaction price
        sigma (float) -- standard deviation of transaction price
        lower_bound (float) -- distribution lower bound
        upper_bound (float) -- distribution upper bound
        """
        if lower_bound > upper_bound:
            raise ValueError("creation: lower bound value can't be bigger than upper bound")
        elif mu < lower_bound:
            raise ValueError("creation: mean value (mu) can't be lower than lower bound")
        elif mu > upper_bound:
            raise ValueError("creation: mean value (mu) can't be bigger than upper bound")
        
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.mu = mu
        self.sigma = sigma
      
    
    def reset_mean_and_dev(self, mu: float, sigma: float):
        """
        Change mean and standard deviation parameters of existing generator
        
        Keyword_arguments:
        mu (float) -- mean transaction price
        sigma (float) -- standard deviation of transaction price
        """
        if mu < self.lower_bound:
            raise ValueError("editing mu and sigma: mean value (mu) can't be lower than lower bound")
        elif mu > self.upper_bound:
            raise ValueError("editing mu and sigma: mean value (mu) can't be bigger than upper bound")
        
        self.mu = mu
        self.sigma = sigma
        
        
    def reset_lower_and_upper_bound(self, lower_bound: float, upper_bound: float):
        """
        Change distribution lower and upper bounds
        
        Keyword arguments:
        lower_bound (float) -- distribution lower bound
        upper_bound (float) -- distribution upper bound
        """
        if lower_bound > upper_bound:
            raise ValueError("bounds editing: lower bound value can't be bigger than upper bound")
        elif self.mu < lower_bound:
            raise ValueError("bounds editing: mean value (mu) can't be lower than lower bound")
        elif self.mu > upper_bound:
            raise ValueError("bounds editing: mean value (mu) can't be bigger than upper bound")
        
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        
    
    def generate_transactions(self, transactions_count: int) -> list:
        """
        Generate transaction prices list considering mu and sigma
        
        Keyword_arguments:
        transactions_count (int) -- required transactions count
        """
        return truncnorm.rvs(
            (self.lower_bound - self.mu)/self.sigma, 
            (self.upper_bound - self.mu)/self.sigma, 
            loc=self.mu, scale=self.sigma, size=transactions_count
        )



class Transaction:
    """
    Class with information regarding swapping transaction
    """
    def __init__(self, timestamp: datetime, 
                token_in_amount: float, token_in: str, 
                token_out: str, 
                slope: float=0.05):
        self.datetime_timestamp = timestamp
        self.token_in = token_in
        self.token_in_amount = token_in_amount
        self.token_out = token_out
        self.token_out_amount = None
        self.slope = slope



    def set_token_out_amount(self, token_out_amount: float):
        self.token_out_amount = token_out_amount
        
    
    def to_string(self) -> str:
        """
        form a string that can be used for showing information on console
        """
        return str(
            'Transaction {datetime timestamp = ' + str(self.datetime_timestamp) + 
            ', token in amount = "' + str(self.token_in_amount) + 
            '", token in name = ' + str(self.token_in) + 
            ', token out amount = ' + str(self.token_out_amount) + 
            ', token out name = ' + str(self.token_out) + 
            ', slope = ' + str(self.slope) + '}'
        )

In [ ]:
class MonteCarloTransactionSimulator:
    """
    Monte Carlo transactions generator, that generates transactions frequency using Poisson 
    distribution and transaction values using normal distribution (time metrics - milliseconds)
    """
    def __init__(
        self, frequency_generator: PoissonGenerator, token_in_generator: NormalGenerator, 
        first_currency: str, second_currency: str
    ):
        """
        Create a Monte-Carlo transaction simulator
        
        Keyword_arguments:
        frequency_generator (PoissonGenerator) -- Poisson transaction distribution generator
        token_in_generator (NormalGenerator) -- Normal transaction distribution generator
        first_currency -- name of the first token in transaction
        second_currency -- name of the second token in transaction
        """
        self.frequency_generator = frequency_generator
        self.token_in_generator = token_in_generator
        self.first_currency = first_currency
        self.second_currency = second_currency
        self.transaction_history = []
        
    
    def reset_values_generator_params(self, mu: float, sigma: float):
        """
        Change parameters of existing values (normal distribution) generator
        
        Keyword_arguments:
        mu (float) -- mean transaction price (default 0.0)
        sigma (float) -- standard deviation of transaction price (default 0.0)
        """
        self.frequency_generator.reset_params(mu, sigma)
        
        
    def reset_values_generator_bounds(self, lower_bound: float, upper_bound: float):
        self.token_in_genererator.reset_lower_and_upper_bound(lower_bound=lower_bound, upper_bound=upper_bound)
        
        
    def clear_transaction_history(self):
        """
        Clear all records from transaction history
        """
        self.transaction_history.clear()
    
    
    def generate_transactions(self, current_timestamp: datetime):
        """
        Generates transactions list with timestamps and token_in values. All transactions are recorded
        to the 'transaction_history' variable.
        
        Keyword arguments:
        current_timestamp (datetime) -- initial datetime point from where cycle will be reviewed
        current_amm_coef (float) -- current AMM market coefficient that defines price of out
            token relative to in token
        """
        # generate timestamps and token_in values
        timestamps = self.frequency_generator.generate_transactions(current_timestamp)
        timestamps.sort()
        token_in_values = self.token_in_generator.generate_transactions(len(timestamps))
        
        # form new transactions and record them into 'transaction history' variable
        for index in range(len(timestamps)):
            new_transaction = Transaction(
                timestamp=timestamps[index], 
                token_in_amount=token_in_values[index], 
                token_in=self.first_currency,
                token_out=self.second_currency
            )
            self.transaction_history.append(new_transaction)


    def get_history(self) -> list:
        """
        Get transaction history
        """
        return self.transaction_history
            
            
    def transaction_history_to_csv(self, filename: str):
        """
        Write transaction history to specified .csv file
        
        Keyword arguments:
        filename (str) -- name of .csv file where to write data
        """
        # form empty dataframe
        transaction_history_dataframe = pd.DataFrame(columns=[
            'datetime_timestamp', 'TokenIn', 'TokenInAmount', 'TokenOut', 'TokenOutAmount', 'Slope'
        ])
        
        # append all new records to the dataframe
        for index in range(len(self.transaction_history)):
            new_row = {
                'datetime_timestamp': self.transaction_history[index].datetime_timestamp,
                'TokenIn': self.transaction_history[index].token_in,
                'TokenInAmount': self.transaction_history[index].token_in_amount, 
                'TokenOut': self.transaction_history[index].token_out,
                'TokenOutAmount': self.transaction_history[index].token_out_amount,
                'Slope': self.transaction_history[index].slope
            }
            transaction_history_dataframe = transaction_history_dataframe.append(new_row, ignore_index=True)
        
        # either append new records to the existing file, or create a new one from existing table
        try:
            with open(filename) as f:
                transaction_history_dataframe.to_csv(filename, mode='a', header=False)
        except IOError:
            transaction_history_dataframe.to_csv(filename)

In [ ]:
# create simulator with specifying parameters of the normal distribution for token_in
# value estimation and Poisson distribution for transaction frequency estimation
simulator = MonteCarloTransactionSimulator(
    PoissonGenerator(cycle_size=60000, mean_occurencies=500), 
    NormalGenerator(mu=10, sigma=5, lower_bound=5, upper_bound=15), 'ETH', 'DAI'
)

# set current timestamp as starting point and start loop, where each iteration shifts reviewable
# timestamp further conform simulator cycle size
current_iteration_timestamp = datetime.now()
for index in range(60*24*7):
    simulator.generate_transactions(current_iteration_timestamp)
    current_iteration_timestamp += timedelta(milliseconds=randrange(simulator.frequency_generator.cycle_size))

# show generated transactions history
# for index in range(len(simulator.transaction_history)):
#     print(simulator.transaction_history[index].to_string())
    
# write generated transactions history to csv file
# simulator.transaction_history_to_csv('history.csv')

In [ ]:
# show generated transactions history
for index in range(int(len(simulator.transaction_history) / 10)):
    print(simulator.transaction_history[index].to_string())

# Fixing timestamp error

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime, timedelta
from scipy.stats import poisson, truncnorm
from random import randrange
import pandas as pd
from scipy.stats import halfcauchy 

In [2]:
class PoissonGenerator:
    """
    Generate transactions timestamps list. Current version works with milliseconds.
    """
    def __init__(self, cycle_size: int, mean_occurencies: float):
        """
        Create a Poisson generator
        
        Keyword_arguments:
        cycle_size (int) -- reviewable cycle size in milliseconds
        mean_occurencies (float) -- mean amount of transactions
            happening in given cycle size
        """
        self.cycle_size = cycle_size
        self.mean_occurencies = mean_occurencies
        self.cumulative_probabilities = 0
        
        
    def __generate_poisson_outcome__(self) -> int:
        """
        Generate how many transactions will happen, considering cycle size 
        and mean transaction amount
        """
        return np.random.poisson(self.mean_occurencies, 1)[0]
    
    
    def generate_transactions(self, current_timestamp: datetime) -> list:
        """
        Generate array of transactions timestamps, considering Poisson distribution and
        random distribution of timestamps in given cycle size
        
        Keyword_arguments:
        current_timestamp (datetime) -- reviewable cycle starting point
        """
        transactions_timestamps = []
        current_transaction_count = self.__generate_poisson_outcome__()
        for i in range(current_transaction_count):
            transactions_timestamps.append(current_timestamp + timedelta(milliseconds=randrange(self.cycle_size)))
        return transactions_timestamps

In [3]:
class NormalGenerator:
    """
    Generate transaction values conform normal distribution
    """
    def __init__(self, mu: float, sigma: float, lower_bound: float, upper_bound: float):
        """
        Create a normal distribution generator
        
        Keyword_arguments:
        mu (float) -- mean transaction price
        sigma (float) -- standard deviation of transaction price
        lower_bound (float) -- distribution lower bound
        upper_bound (float) -- distribution upper bound
        """
        if lower_bound > upper_bound:
            raise ValueError("creation: lower bound value can't be bigger than upper bound")
        elif mu < lower_bound:
            raise ValueError("creation: mean value (mu) can't be lower than lower bound")
        elif mu > upper_bound:
            raise ValueError("creation: mean value (mu) can't be bigger than upper bound")
        
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.mu = mu
        self.sigma = sigma
      
    
    def reset_params(self, mu: float, sigma: float, lower_bound: float, upper_bound: float):
        """
        Change mean and standard deviation parameters of existing generator
        
        Keyword_arguments:
        mu (float) -- mean transaction price
        sigma (float) -- standard deviation of transaction price
        lower_bound (float) -- distribution lower bound
        upper_bound (float) -- distribution upper bound
        """
        if lower_bound > upper_bound:
            raise ValueError("bounds editing: lower bound value can't be bigger than upper bound")
        elif mu < lower_bound:
            raise ValueError("bounds editing: mean value (mu) can't be lower than lower bound")
        elif mu > upper_bound:
            raise ValueError("bounds editing: mean value (mu) can't be bigger than upper bound")
        
        self.mu = mu
        self.sigma = sigma
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        
    
    def generate_transactions(self, transactions_count: int) -> list:
        """
        Generate transaction prices list considering mu and sigma
        
        Keyword_arguments:
        transactions_count (int) -- required transactions count
        """
        return truncnorm.rvs(
            (self.lower_bound - self.mu)/self.sigma, 
            (self.upper_bound - self.mu)/self.sigma, 
            loc=self.mu, scale=self.sigma, size=transactions_count
        )

In [17]:
class Transaction:
    """
    Class with information regarding swapping transaction
    """
    def __init__(self, timestamp: datetime, 
                token_in_amount: float, token_in: str, 
                token_out: str, 
                slope: float=0.05):
        self.datetime_timestamp = timestamp
        self.token_in = token_in
        self.token_in_amount = token_in_amount
        self.token_out = token_out
        self.token_out_amount = None
        self.slope = slope


    def set_token_out_amount(self, token_out_amount: float):
        """
        set token out value for transaction

        Keyword arguments:
        token_out_amount (float) -- amount of out_token in transaction
        """
        self.token_out_amount = token_out_amount
        
    
    def to_string(self) -> str:
        return str(
            'Transaction {datetime timestamp = ' + str(self.datetime_timestamp) + 
            ', token in amount = "' + str(self.token_in_amount) + 
            '", token in name = ' + str(self.token_in) + 
            ', token out amount = ' + str(self.token_out_amount) + 
            ', token out name = ' + str(self.token_out) + 
            ', slope = ' + str(self.slope) + '}'
        )

    
    def to_record(self) -> np.array:
        """
        Transform transaction data into numpy array of data
        """
        return np.array([
            self.datetime_timestamp,
            self.token_in,
            self.token_in_amount,
            self.token_out,
            self.token_out_amount,
            self.slope
        ])

In [ ]:
        """
        self.token_out_amount = token_out_amount
        
    
    def to_string(self) -> str:
        return str(
            'Transaction {datetime timestamp = ' + str(self.datetime_timestamp) + 
            ', token in amount = "' + str(self.token_in_amount) + 
            '", token in name = ' + str(self.token_in) + 
            ', token out amount = ' + str(self.token_out_amount) + 
            ', token out name = ' + str(self.token_out) + 
            ', slope = ' + str(self.slope) + '}'
        )

    
    def to_record(self) -> np.array:
        """
        Transform transaction data into numpy array of data
        """
        return np.array([
            self.datetime_timestamp,
            self.token_in,
            self.token_in_amount,
            self.token_out,
            self.token_out_amount,
            self.slope
        ])

In [5]:
class MonteCarloTransactionSimulator:
    """
    Monte Carlo transactions generator, that generates transactions frequency using Poisson 
    distribution and transaction values using normal distribution (time metrics - milliseconds)
    """
    def __init__(
        self, frequency_generator: PoissonGenerator, token_in_generator, 
        first_currency: str, second_currency: str
    ):
        """
        Create a Monte-Carlo transaction simulator
        
        Keyword_arguments:
        frequency_generator (PoissonGenerator) -- Poisson transaction distribution generator
        token_in_generator -- transaction distribution generator (accepts normal, Cauchy and 
            Pareto)
        first_currency -- name of the first token in transaction
        second_currency -- name of the second token in transaction
        """
        self.frequency_generator = frequency_generator
        self.token_in_generator = token_in_generator
        self.first_currency = first_currency
        self.second_currency = second_currency
        self.transaction_history = []
        
    
    def get_token_in_generator(self):
        """
        Get transactions distribution generator (can be normal, Cauchy and Pareto)
        """
        return self.token_in_generator
        
        
    def clear_transaction_history(self):
        """
        Clear all records from transaction history
        """
        self.transaction_history.clear()
    
    
    def generate_transactions(self, current_timestamp: datetime):
        """
        Generates transactions list with timestamps and token_in values. All transactions are recorded
        to the 'transaction_history' variable.
        
        Keyword arguments:
        current_timestamp (datetime) -- initial datetime point from where cycle will be reviewed
        current_amm_coef (float) -- current AMM market coefficient that defines price of out
            token relative to in token
        """
        # generate timestamps and token_in values
        timestamps = self.frequency_generator.generate_transactions(current_timestamp)
        timestamps.sort()
        token_in_values = self.token_in_generator.generate_transactions(len(timestamps))
        
        # form new transactions and record them into 'transaction history' variable
        for index in range(len(timestamps)):
            new_transaction = Transaction(
                timestamp=timestamps[index], 
                token_in_amount=token_in_values[index], 
                token_in=self.first_currency,
                token_out=self.second_currency
            )
            self.transaction_history.append(new_transaction)


    def get_history(self) -> list:
        """
        Get transaction history
        """
        return self.transaction_history
            
            
    def transaction_history_to_csv(self, filename: str):
        """
        Write transaction history to specified .csv file
        
        Keyword arguments:
        filename (str) -- name of .csv file where to write data
        """
        # vectorize all transactions into numpy matrix and then make dataframe out of it
        transactions_matrix = np.array([transaction.to_record() for transaction in self.transaction_history])
        transaction_history_df = pd.DataFrame(data=transactions_matrix, columns=[
            'datetime_timestamp', 'TokenIn', 'TokenInAmount', 'TokenOut', 'TokenOutAmount', 'Slope'
        ])

        # fix transformation of numerical features into string at numpy stage to numerical again
        transaction_history_df['TokenInAmount'] = pd.to_numeric(transaction_history_df['TokenInAmount'])
        transaction_history_df['TokenOutAmount'] = pd.to_numeric(transaction_history_df['TokenOutAmount'])
        
        # either append new records to the existing file, or create a new one from existing table
        try:
            with open(filename) as f:
                transaction_history_df.to_csv(filename, mode='a', header=False)
        except IOError:
            transaction_history_df.to_csv(filename)


    def get_dataframe(self) -> pd.DataFrame:
        # vectorize all transactions into numpy matrix and then make dataframe out of it
        transactions_matrix = np.array([transaction.to_record() for transaction in self.transaction_history])
        transaction_history_df = pd.DataFrame(data=transactions_matrix, columns=[
            'datetime_timestamp', 'TokenIn', 'TokenInAmount', 'TokenOut', 'TokenOutAmount', 'Slope'
        ])

        # fix transformation of numerical features into string at numpy stage to numerical again
        transaction_history_df['TokenInAmount'] = pd.to_numeric(transaction_history_df['TokenInAmount'])
        transaction_history_df['TokenOutAmount'] = pd.to_numeric(transaction_history_df['TokenOutAmount'])

        return transaction_history_df

In [16]:
# several simulators, where each uses its unique values generator
normal_simulator = MonteCarloTransactionSimulator(
    PoissonGenerator(cycle_size=60000, mean_occurencies=2), 
    NormalGenerator(mu=0, sigma=4500, lower_bound=0, upper_bound=10000), 'ETH', 'DAI'
)

# set current timestamp as starting point and start loop, where each iteration shifts reviewable
# timestamp further conform simulator cycle size
current_iteration_timestamp = datetime.now()

for index in range(10080):
    print(current_iteration_timestamp)
    normal_simulator.generate_transactions(current_iteration_timestamp)
    current_iteration_timestamp += timedelta(milliseconds=normal_simulator.frequency_generator.cycle_size)

current moment >>>>    2021-10-29 11:40:13.179544
current moment >>>>    2021-10-29 11:40:13.180544
2021-10-29 11:40:13.179544
2021-10-29 11:41:13.179544
2021-10-29 11:42:13.179544
2021-10-29 11:43:13.179544
2021-10-29 11:44:13.179544
2021-10-29 11:45:13.179544
2021-10-29 11:46:13.179544
2021-10-29 11:47:13.179544
2021-10-29 11:48:13.179544
2021-10-29 11:49:13.179544
2021-10-29 11:50:13.179544
2021-10-29 11:51:13.179544
2021-10-29 11:52:13.179544
2021-10-29 11:53:13.179544
2021-10-29 11:54:13.179544
2021-10-29 11:55:13.179544
2021-10-29 11:56:13.179544
2021-10-29 11:57:13.179544
2021-10-29 11:58:13.179544
2021-10-29 11:59:13.179544
2021-10-29 12:00:13.179544
2021-10-29 12:01:13.179544
2021-10-29 12:02:13.179544
2021-10-29 12:03:13.179544
2021-10-29 12:04:13.179544
2021-10-29 12:05:13.179544
2021-10-29 12:06:13.179544
2021-10-29 12:07:13.179544
2021-10-29 12:08:13.179544
2021-10-29 12:09:13.179544
2021-10-29 12:10:13.179544
2021-10-29 12:11:13.179544
2021-10-29 12:12:13.179544
2021-10-2